<a href="https://colab.research.google.com/github/CristianoDataScience/Projeto_Aplicado_SRAG/blob/main/Classificacao_Modelo_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importando Bibliotecas

In [15]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


## Importando base de dados

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Projeto_Aplicado_SRAG/base_dados_limpa_srag_sp.csv', sep=',',encoding='iso-8859-1')
df

,data,municipio,faixa_etaria,sexo,srag,evolucao,qtde_masculino,qtde_feminino,total
0,2023-02-09,SAO PAULO,10,F,Outras SRAG,Cura,0,1,1
1,2023-02-24,SAO PAULO,10,M,Outras SRAG,Em Avaliação,1,0,1
2,2021-08-13,BRAGANCA PAULISTA,10,F,Outras SRAG,Cura,0,1,1
3,2023-02-19,PRAIA GRANDE,10,F,Outras SRAG,Em Avaliação,0,1,1
4,2023-02-22,SAO JOSE DO RIO PRETO,20,F,Outras SRAG,Cura,0,1,1
...,...,...,...,...,...,...,...,...,...
346738,2020-07-24,CAMPINAS,80,M,Outras SRAG,Cura,1,0,1
346739,2020-07-20,MOGI DAS CRUZES,45,M,Outras SRAG,Óbito,1,0,1
346740,2020-06-01,GOIANIA,0,F,Outras SRAG,Cura,0,1,1
346741,2020-04-16,SAO PAULO,60,M,Outras SRAG,Óbito,1,0,1


In [ ]:
# Análise para cada coluna do DataFrame
df = df[['faixa_etaria', 'sexo', 'qtde_masculino', 'qtde_feminino', 'total', 'evolucao']]
df

,faixa_etaria,sexo,qtde_masculino,qtde_feminino,total,evolucao
0,10,F,0,1,1,Cura
1,10,M,1,0,1,Em Avaliação
2,10,F,0,1,1,Cura
3,10,F,0,1,1,Em Avaliação
4,20,F,0,1,1,Cura
...,...,...,...,...,...,...
346738,80,M,1,0,1,Cura
346739,45,M,1,0,1,Óbito
346740,0,F,0,1,1,Cura
346741,60,M,1,0,1,Óbito


In [16]:
pio.templates.default = "plotly_dark"

df = df[['faixa_etaria', 'sexo', 'qtde_masculino', 'qtde_feminino', 'total', 'evolucao']]
fig = px.histogram(df, x='faixa_etaria', y='total', color='evolucao', barmode='group', title='Casos por Faixa Etária')
fig.show()


In [17]:
!pip3 install dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 54.4 MB/s eta 0:00:00


In [ ]:
df = df[df['evolucao'] != 'Em Avaliação']

In [ ]:
#df = df[(df['municipio'] == 'GUARULHOS') | (df['municipio'] == 'SAO PAULO')]

In [ ]:
df2 = pd.DataFrame(df)

In [ ]:
df2['evolucao'].value_counts()

Cura     233436
Óbito     45602
Name: evolucao, dtype: int64

## Atributos Previsores e Alvo
___

### Codificação de variáveis categóricas

### LabelEncoder: Transformação de variáveis categóricas em numéricas

In [ ]:
df2.head(2)

,faixa_etaria,sexo,qtde_masculino,qtde_feminino,total,evolucao
0,10,F,0,1,1,Cura
2,10,F,0,1,1,Cura


In [ ]:
previsores = df2.iloc[:, 0:5].values

In [ ]:
previsores

array([[10, 'F', 0, 1, 1],
       [10, 'F', 0, 1, 1],
       [20, 'F', 0, 1, 1],
       ...,
       [0, 'F', 0, 1, 1],
       [60, 'M', 1, 0, 1],
       [65, 'F', 0, 1, 1]], dtype=object)

In [ ]:
previsores[:, 1] = LabelEncoder().fit_transform(previsores[:,1])
#previsores[:, 2] = LabelEncoder().fit_transform(previsores[:,2])

In [ ]:
alvo = df2.iloc[:, 5].values
alvo

array(['Cura', 'Cura', 'Cura', ..., 'Cura', 'Óbito', 'Cura'], dtype=object)

In [ ]:
alvo = LabelEncoder().fit_transform(alvo)
alvo

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
previsores

array([[10, 0, 0, 1, 1],
       [10, 0, 0, 1, 1],
       [20, 0, 0, 1, 1],
       ...,
       [0, 0, 0, 1, 1],
       [60, 1, 1, 0, 1],
       [65, 0, 0, 1, 1]], dtype=object)

## OneHotEncoder: Criação de variáveis Dummy

In [ ]:
previsores2 = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [1])],
                                remainder='passthrough').fit_transform(previsores)

In [ ]:
previsores2

array([[1.0, 0.0, 0.0, ..., 0, 1, 1],
       [1.0, 0.0, 0.0, ..., 0, 1, 1],
       [1.0, 0.0, 0.0, ..., 0, 1, 1],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 1],
       [0.0, 1.0, 0.0, ..., 1, 0, 1],
       [1.0, 0.0, 0.0, ..., 0, 1, 1]], dtype=object)

In [ ]:
previsores2df = pd.DataFrame(previsores2)
previsores2df

,0,1,2,3,4,5,6
0,1.0,0.0,0.0,10,0,1,1
1,1.0,0.0,0.0,10,0,1,1
2,1.0,0.0,0.0,20,0,1,1
3,0.0,1.0,0.0,85,1,0,1
4,1.0,0.0,0.0,90,0,2,2
...,...,...,...,...,...,...,...
279033,0.0,1.0,0.0,80,1,0,1
279034,0.0,1.0,0.0,45,1,0,1
279035,1.0,0.0,0.0,0,0,1,1
279036,0.0,1.0,0.0,60,1,0,1


In [ ]:
previsores2_esc = StandardScaler().fit_transform(previsores2)
previsores2_esc

array([[ 1.02424762, -1.0241301 , -0.00757253, ..., -0.77516209,
         0.57368678, -0.26152029],
       [ 1.02424762, -1.0241301 , -0.00757253, ..., -0.77516209,
         0.57368678, -0.26152029],
       [ 1.02424762, -1.0241301 , -0.00757253, ..., -0.77516209,
         0.57368678, -0.26152029],
       ...,
       [ 1.02424762, -1.0241301 , -0.00757253, ..., -0.77516209,
         0.57368678, -0.26152029],
       [-0.97632641,  0.97643845, -0.00757253, ...,  0.50243463,
        -0.76935984, -0.26152029],
       [ 1.02424762, -1.0241301 , -0.00757253, ..., -0.77516209,
         0.57368678, -0.26152029]])

In [ ]:
previsores3df = pd.DataFrame(previsores2_esc)
previsores3df

,0,1,2,3,4,5,6
0,1.024248,-1.024130,-0.007573,-1.178337,-0.775162,0.573687,-0.261520
1,1.024248,-1.024130,-0.007573,-1.178337,-0.775162,0.573687,-0.261520
2,1.024248,-1.024130,-0.007573,-0.845287,-0.775162,0.573687,-0.261520
3,-0.976326,0.976438,-0.007573,1.319533,0.502435,-0.769360,-0.261520
4,1.024248,-1.024130,-0.007573,1.486057,-0.775162,1.916733,1.194293
...,...,...,...,...,...,...,...
279033,-0.976326,0.976438,-0.007573,1.153008,0.502435,-0.769360,-0.261520
279034,-0.976326,0.976438,-0.007573,-0.012664,0.502435,-0.769360,-0.261520
279035,1.024248,-1.024130,-0.007573,-1.511386,-0.775162,0.573687,-0.261520
279036,-0.976326,0.976438,-0.007573,0.486910,0.502435,-0.769360,-0.261520


## Análise das escalas dos atributos (Escalonamento)

### Padronização (utiliza a média e o desvio padrão como referência).
### Normalização (utiliza os valores máximo e mínimo como referência).

In [ ]:
previsores_esc = StandardScaler().fit_transform(previsores2)
previsores_esc

array([[ 1.02424762, -1.0241301 , -0.00757253, ..., -0.77516209,
         0.57368678, -0.26152029],
       [ 1.02424762, -1.0241301 , -0.00757253, ..., -0.77516209,
         0.57368678, -0.26152029],
       [ 1.02424762, -1.0241301 , -0.00757253, ..., -0.77516209,
         0.57368678, -0.26152029],
       ...,
       [ 1.02424762, -1.0241301 , -0.00757253, ..., -0.77516209,
         0.57368678, -0.26152029],
       [-0.97632641,  0.97643845, -0.00757253, ...,  0.50243463,
        -0.76935984, -0.26152029],
       [ 1.02424762, -1.0241301 , -0.00757253, ..., -0.77516209,
         0.57368678, -0.26152029]])

In [ ]:
previsoresdf = pd.DataFrame(previsores_esc)
previsoresdf

,0,1,2,3,4,5,6
0,1.024248,-1.024130,-0.007573,-1.178337,-0.775162,0.573687,-0.261520
1,1.024248,-1.024130,-0.007573,-1.178337,-0.775162,0.573687,-0.261520
2,1.024248,-1.024130,-0.007573,-0.845287,-0.775162,0.573687,-0.261520
3,-0.976326,0.976438,-0.007573,1.319533,0.502435,-0.769360,-0.261520
4,1.024248,-1.024130,-0.007573,1.486057,-0.775162,1.916733,1.194293
...,...,...,...,...,...,...,...
279033,-0.976326,0.976438,-0.007573,1.153008,0.502435,-0.769360,-0.261520
279034,-0.976326,0.976438,-0.007573,-0.012664,0.502435,-0.769360,-0.261520
279035,1.024248,-1.024130,-0.007573,-1.511386,-0.775162,0.573687,-0.261520
279036,-0.976326,0.976438,-0.007573,0.486910,0.502435,-0.769360,-0.261520


In [ ]:
previsoresdf.describe()

,0,1,2,3,4,5,6
count,2.790380e+05,2.790380e+05,2.790380e+05,2.790380e+05,2.790380e+05,2.790380e+05,2.790380e+05
mean,-8.596651e-17,-7.398569e-17,-1.101319e-18,7.470505e-17,-1.632243e-17,-5.428928e-17,1.393900e-16
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-9.763264e-01,-1.024130e+00,-7.572526e-03,-1.511386e+00,-7.751621e-01,-7.693598e-01,-2.615203e-01
25%,-9.763264e-01,-1.024130e+00,-7.572526e-03,-1.011812e+00,-7.751621e-01,-7.693598e-01,-2.615203e-01
50%,-9.763264e-01,9.764384e-01,-7.572526e-03,1.538603e-01,5.024346e-01,-7.693598e-01,-2.615203e-01
75%,1.024248e+00,9.764384e-01,-7.572526e-03,8.199588e-01,5.024346e-01,5.736868e-01,-2.615203e-01
max,1.024248e+00,9.764384e-01,1.320563e+02,2.318680e+00,2.222158e+01,2.877767e+01,3.031056e+01


## Base de Treino e Teste

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(previsores, alvo, test_size= 0.3, random_state=0)

In [ ]:
x_treino.shape

(195326, 5)

In [ ]:
x_teste.shape

(83712, 5)

In [ ]:
y_treino.shape

(195326,)

In [ ]:
y_teste.shape

(83712,)

## Modelo Naive Bayes
___

* Treinamento do algoritmo

In [ ]:
naive = GaussianNB()
naive.fit(x_treino, y_treino)

GaussianNB()

## Avaliação do algoritmo

In [ ]:
previsoes_naive = naive.predict(x_teste)
previsoes_naive

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
y_teste

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
accuracy_score(y_teste, previsoes_naive)

0.6483658256880734

In [ ]:
print("Acurácia: %.2f%%" % (accuracy_score(y_teste, previsoes_naive) * 100.0))

Acurácia: 64.84%


In [ ]:
confusion_matrix(y_teste, previsoes_naive)

array([[44331, 25740],
       [ 3696,  9945]])

In [ ]:
print(classification_report(y_teste, previsoes_naive))

              precision    recall  f1-score   support

           0       0.92      0.63      0.75     70071
           1       0.28      0.73      0.40     13641

    accuracy                           0.65     83712
   macro avg       0.60      0.68      0.58     83712
weighted avg       0.82      0.65      0.69     83712



## Análise dados treino

In [ ]:
previsoes_treino = naive.predict(x_treino)
previsoes_treino

array([0, 0, 0, ..., 1, 0, 0])

In [ ]:
accuracy_score(y_treino, previsoes_treino)

0.6470413565014386

In [ ]:
confusion_matrix(y_treino, previsoes_treino)

array([[102981,  60384],
       [  8558,  23403]])

## Validação Cruzada

In [ ]:
# Separando os dados em folds
kfold = KFold(n_splits = 30, shuffle = True, random_state = 5)

In [ ]:
# Criando o modelo
modelo = GaussianNB()
resultado = cross_val_score(modelo, previsores, alvo, cv = kfold)

In [ ]:
# Usaremos a média e o desvio padrão
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

Acurácia Média: 64.74%


## Regressão Logística

In [ ]:
logistica = LogisticRegression(random_state=1, max_iter=500, penalty="l2", tol=0.0001, C=10, solver='lbfgs', l1_ratio=0.5)
logistica.fit(x_treino, y_treino)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


LogisticRegression(C=10, l1_ratio=0.5, max_iter=500, random_state=1)

In [ ]:
logistica.intercept_

array([-3.18931898])

In [ ]:
logistica.coef_

array([[ 0.03566266,  0.27201383, -0.17953881, -0.15260243, -0.33503126]])

In [ ]:
previsoes_logistica = logistica.predict(x_teste)
previsoes_logistica

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
print("Acurácia: %.2f%%" % (accuracy_score(y_teste, previsoes_logistica) * 100.0))

Acurácia: 83.70%


In [ ]:
confusion_matrix(y_teste, previsoes_logistica)

array([[70047,    24],
       [13623,    18]])

In [ ]:
print(classification_report(y_teste, previsoes_logistica))

              precision    recall  f1-score   support

           0       0.84      1.00      0.91     70071
           1       0.43      0.00      0.00     13641

    accuracy                           0.84     83712
   macro avg       0.63      0.50      0.46     83712
weighted avg       0.77      0.84      0.76     83712



## Análise dados de treino

In [ ]:
previsoes_treino_logistica = logistica.predict(x_treino)
previsoes_treino_logistica

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
accuracy_score(y_treino, previsoes_treino_logistica)

0.8362993149913478

In [ ]:
confusion_matrix(y_treino, previsoes_treino_logistica)

array([[163309,     56],
       [ 31919,     42]])

In [ ]:
modelo = LogisticRegression(random_state=2, max_iter=500, penalty="l2", tol=0.0001, C=1, solver="lbfgs")
resultado = cross_val_score(modelo, previsores, alvo, cv=kfold)

In [ ]:
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

Acurácia Média: 83.65%


## Aprendizagem Random Forest

In [ ]:
random = RandomForestClassifier(n_estimators=1500, criterion='entropy', random_state= 5, max_depth=9)
random.fit(x_treino, y_treino)

RandomForestClassifier(criterion='entropy', max_depth=9, n_estimators=1500,
                       random_state=5)

In [ ]:
previsoes_random = random.predict(x_teste)
previsoes_random

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
y_teste

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
print("Acurácia: %.2f%%" % ( accuracy_score(y_teste, previsoes_random) * 100.0))

Acurácia: 83.70%


## MODELO LIGHTGBM

In [ ]:
import lightgbm as lgb

In [ ]:
dataset = lgb.Dataset(x_treino, label=y_treino)

## Hiperparâmetros

In [ ]:
# Parâmetros
parametros = {'num_leaves': 500, # Número de folhas
              'objective': 'binary', # classificação Binária
              'max_depth': 5,
              'learning_rate': .05,
              'max_bin': 900}

In [ ]:
lgbm = lgb.train(parametros, dataset, num_boost_round=500)

[LightGBM] [Info] Number of positive: 31961, number of negative: 163365
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012884 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 78
[LightGBM] [Info] Number of data points in the train set: 195326, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.163629 -> initscore=-1.631471
[LightGBM] [Info] Start training from score -1.631471
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

In [ ]:
# Marcação de tempo de execução
from datetime import datetime
inicio = datetime.now()
lgbm = lgb.train(parametros, dataset)
fim = datetime.now()

tempo = fim - inicio
tempo

[LightGBM] [Info] Number of positive: 31961, number of negative: 163365
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009247 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 78
[LightGBM] [Info] Number of data points in the train set: 195326, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.163629 -> initscore=-1.631471
[LightGBM] [Info] Start training from score -1.631471
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

datetime.timedelta(seconds=1, microseconds=438688)

In [ ]:
previsoes_lgbm = lgbm.predict(x_teste)
previsoes_lgbm

array([0.0178534 , 0.1952862 , 0.00487265, ..., 0.01946856, 0.31296853,
       0.0178534 ])

In [ ]:
previsoes_lgbm.shape

(83712,)

In [ ]:
# Quando for menor que 5 considera 0 e quando for maior ou igual a 5 considera 1
for i in range(0, 83712):
  if previsoes_lgbm[i] >= .5:
    previsoes_lgbm[i] = 1
  else:
    previsoes_lgbm[i] = 0

In [ ]:
previsoes_lgbm

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
y_teste

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
print("Acurácia: %.2f%%" % ( accuracy_score(y_teste, previsoes_lgbm) * 100.0))

Acurácia: 83.70%


In [ ]:
confusion_matrix(y_teste, previsoes_lgbm)

array([[70071,     0],
       [13641,     0]])

## Análise dados de treino 

In [ ]:
previsoes_treino = lgbm.predict(x_treino)
previsoes_treino

array([0.01946856, 0.05014979, 0.09992292, ..., 0.22307376, 0.0178534 ,
       0.15690305])

In [ ]:
previsoes_treino.shape

(195326,)

In [ ]:
# Quando for menor que 5 considera 0 e quando for maior ou igual a 5 considera 1
for i in range(0, 195326):
  if previsoes_treino[i] >= .5:
    previsoes_treino[i] = 1
  else:
    previsoes_treino[i] = 0

In [ ]:
previsoes_treino

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
accuracy_score(y_treino, previsoes_treino)

0.8363709900371686

In [ ]:
confusion_matrix(y_treino, previsoes_treino)

array([[163365,      0],
       [ 31961,      0]])

In [ ]:
print("Acurácia: %.2f%%" % ( accuracy_score(y_treino, previsoes_treino) * 100.0))

Acurácia: 83.64%
